In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

In [2]:
Window = None
Shader_programm = None
Vao = None
WIDTH = 800
HEIGHT = 600

In [3]:
#Função callback que é executada sempre que a janela for redimensionada
#Sempre que a tela for redimensionada, salvamos sua nova largura e altura nas variáveis globais acima
def redimensionaCallback(window, w, h):
    global WIDTH, HEIGHT
    WIDTH = w
    HEIGHT = h

In [4]:
def inicializaOpenGL():
    global Window, WIDTH, HEIGHT

    #Inicializa GLFW
    glfw.init()

    #Criação de uma janela
    Window = glfw.create_window(WIDTH, HEIGHT, "Exemplo - renderização de um triângulo", None, None)
    #Caso não seja possível criar a janela, a GLFW e a aplicação são terminadas
    if not Window:
        glfw.terminate()
        exit()

    #Registramos a função "redimensionaCallback" como sendo a função de redimensionamento
	#Isso significa que a função "redimensionaCallback" será chamada sempre que a janela for redimensionada,
	#seja pelo sistema ou pelo usuário
    glfw.set_window_size_callback(Window, redimensionaCallback)

    #Define o contexto atual do GLFW como sendo a janela criada acima. O contexto define
	#em qual janela o OpenGL irá funcionar, o que é essencial para que o programa funcione
    glfw.make_context_current(Window)

    #Buscamos informações a respeito do hardware (placa de vídeo) e a versão do OpenGL que a mesma da suporte
    print("Placa de vídeo: ",OpenGL.GL.glGetString(OpenGL.GL.GL_RENDERER))
    print("Versão do OpenGL: ",OpenGL.GL.glGetString(OpenGL.GL.GL_VERSION))

In [5]:
def inicializaObjetos():

    global Vao

	# Geramos o VAO, definindo um identificador para ele através glGenVertexArrays
    Vao = glGenVertexArrays(1)
    # Damos um bind no VAO, setando ele como VAO atual e colocando o mesmo no topo da máquina de estados do OpenGL
    glBindVertexArray(Vao)


    points = [
		-0.5, 0.5, 0.0, #cima
		0.0, -0.5, 0.0, #direita
		-1.0, -0.5, 0.0, #/esquerda
        
        #triângulo retângulo
        0.2, 0.7, 0.0, #cima
        0.2, 0.3, 0.0, #baixo
        0.6, 0.3, 0.0, #lado
        
        #quadrado
        0.2, 0.0, 0.0, #cima
        0.2, -0.5, 0.0, #baixo
        0.6, -0.5, 0.0, #lado
        
        0.6, -0.5, 0.0, #cima
        0.6, 0.0, 0.0, #baixo
        0.2, 0.0, 0.0 #lado
	]

    # Convertemos o array do Python para um array da biblioteca NumPy, pois é o tipo de array que o OpenGL trabalha
    points = np.array(points, dtype=np.float32)

    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    # Ativamos o primeiro atributo do VAO (�ndice 0), que é o atributo referente ao buffer das posições dos vértices.
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)


    # Definição de um VBO para as cores do triângulo. Observe que passamos como parâmetro o valor 1
	# na chamada ao "glEnableVertexAttribArray", pois estamos ativando o segundo atributo deste VAO,
	# que são as cores dos vértices. Além disso, também passamos o parâmetro 1 na chamada ao "glVertexAttribPointer", 
	# pois estamos definindo o layout do segundo atributo.
    cores = [
		1.0, 0.0, 0.0, #vermelho
		0.0, 1.0, 0.0, #verde
		0.0, 0.0, 1.0,  #azul
        # cores ret
        0.0, 1.0, 0.0, #vermelho
		0.0, 0.7, 0.5, #verde
		0.6, 0.7, 0.0, # azul
        # cores quad
        0.7, 0.0, 0.9, #vermelho
		0.7, 0.0, 0.7, #verde
		0.7, 0.0, 0.8,  #azul
        
        0.7, 0.0, 0.9, #vermelho
		0.7, 0.0, 0.7, #verde
		0.7, 0.0, 0.8  #azul
	]
    cores = np.array(cores, dtype=np.float32)
    cvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, cvbo)
    glBufferData(GL_ARRAY_BUFFER, cores, GL_STATIC_DRAW)
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 0, None)

In [6]:
def inicializaTriangulo():

    global Vao

    Vao = glGenVertexArrays(1)
    glBindVertexArray(Vao)
    points = [
		0.4, 0.5, 0.0, #cima (x, y, z)
		0.9, -0.5, 0.0, #direita
		-0.1, -0.5, 0.0 #esquerda
	]

    points = np.array(points, dtype=np.float32)

    pvbo = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, pvbo)
    glBufferData(GL_ARRAY_BUFFER, points, GL_STATIC_DRAW)
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 0, None)


In [7]:
def inicializaShaders():
    global Shader_programm

    vertex_shader = """
        #version 400
        layout(location = 0) in vec3 vertex_posicao;
        layout(location = 1) in vec3 vertex_cores;
        out vec3 cores;
        void main () {
            cores = vertex_cores;
            gl_Position = vec4 (-vertex_posicao, 1.0);
        }
    """
    vs = OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER)
    if not glGetShaderiv(vs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(vs, 512, None)
        print("Erro no vertex shader:\n", infoLog)

    fragment_shader = """
        #version 400
        in vec3 cores;
		out vec4 frag_colour;
		void main () {
		    frag_colour = vec4 (1-cores, 1.0);
		}
    """
    fs = OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs, 512, None)
        print("Erro no fragment shader:\n", infoLog)

    Shader_programm = OpenGL.GL.shaders.compileProgram(vs, fs)
    if not glGetProgramiv(Shader_programm, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_programm, 512, None)
        print("Erro na linkagem do shader:\n", infoLog)

    glDeleteShader(vs)
    glDeleteShader(fs)

In [8]:
def inicializaShaders2():
    global Shader_programm

    vertex_shader = """
        #version 400
        layout(location = 0) in vec3 vertex_posicao; //vem do python
        out vec3 posicor; //vai ser passada para o fragment shader
        void main () {
            posicor = vertex_posicao;
            gl_Position = vec4 (vertex_posicao.x, vertex_posicao.y, vertex_posicao.z, 1.0);
        }
    """
    vs = OpenGL.GL.shaders.compileShader(vertex_shader, GL_VERTEX_SHADER)
    if not glGetShaderiv(vs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(vs, 512, None)
        print("Erro no vertex shader:\n", infoLog)

    fragment_shader = """
        #version 400
        in vec3 posicor; //variavel recebida do vertex shader
		out vec4 frag_colour; //sai do fragment shader e vai para a tela do computador (RGBA)
		void main () {
		    frag_colour = vec4 (posicor, 1.0);
		}
    """
    fs = OpenGL.GL.shaders.compileShader(fragment_shader, GL_FRAGMENT_SHADER)
    if not glGetShaderiv(fs, GL_COMPILE_STATUS):
        infoLog = glGetShaderInfoLog(fs, 512, None)
        print("Erro no fragment shader:\n", infoLog)

    Shader_programm = OpenGL.GL.shaders.compileProgram(vs, fs)
    if not glGetProgramiv(Shader_programm, GL_LINK_STATUS):
        infoLog = glGetProgramInfoLog(Shader_programm, 512, None)
        print("Erro na linkagem do shader:\n", infoLog)

    glDeleteShader(vs)
    glDeleteShader(fs)

In [10]:
def inicializaRenderizacao():
    global Window, Shader_programm, Vao, WIDTH, HEIGHT

	# O triangulo é redesenhado o tempo todo, dentro de um laço de repetição
	# que é executado enquanto a janela não for fechada
    while not glfw.window_should_close(Window):
        # Limpamos o buffer de cores da tela
        glClear(GL_COLOR_BUFFER_BIT)
        glClearColor(0.2, 0.3, 0.3, 1.0)

        # Redefinimos o tamanho da viewport para o tamanho atual da janela, a cada frame, de modo que
		# o desenho se ajuste de acordo com o tamanho da tela
        glViewport(0, 0, WIDTH, HEIGHT)

        # Especificamos qual Shader Programm vamos utilizar
        glUseProgram(Shader_programm)

        # Setamos o objeto Vao como sendo o VAO atual na máquina de estados do OpenGL
        glBindVertexArray(Vao)

        # Desenhamos o triângulo especificado no vao
        glDrawArrays(GL_TRIANGLES, 0, 12)

        # Atualiamos outros eventos, tais como entradas pelo teclado, mouse, etc, caso ocorram
        glfw.poll_events()

        # Renderizamos na tela tudo aquilo que foi desenhado logo acima
        glfw.swap_buffers(Window)

        # Verificamos se a tecla ESC foi pressionada. Caso positivo, definimos que a tela deve ser
		# fechada na próxima volta do laço.
		# Para testar se outras teclas foram pressionadas, verifique o seguinte link:
		# http://www.glfw.org/docs/latest/group__input.html
        if (glfw.PRESS == glfw.get_key(Window, glfw.KEY_ESCAPE)):
            glfw.set_window_should_close(Window, True)
    
    glfw.terminate()

In [11]:
# Função principal
def main():
    inicializaOpenGL()
    inicializaObjetos()
    inicializaShaders()
    inicializaRenderizacao()

In [12]:
# Função exercício 2
def main2():
    inicializaOpenGL()
    inicializaTriangulo()
    inicializaShaders2()
    inicializaRenderizacao()

In [14]:
# Print por Gabriel D. F. Mello
#main()

In [15]:
main2()

Placa de vídeo:  b'Intel(R) HD Graphics 630'
Versão do OpenGL:  b'4.6.0 - Build 26.20.100.8142'
